让我们讨论一下如何在 PyTorch 中表达模型。

In [ ]:
import torch                     # 导入 PyTorch 核心库
import torch.nn as nn            # 导入神经网络模块，nn.Module 是 PyTorch 模型的父对象
import torch.nn.functional as F  # 导入神经网络函数库，通常用于激活函数等无状态操作

<img src="lenet5.png">

*图：LeNet-5*

上图是 LeNet-5 的示意图，它是最早的卷积神经网络之一，也是推动深度学习爆炸式发展的驱动力之一。它被构建用于读取手写数字的小图像（MNIST 数据集），并正确分类图像中表示的数字。

以下是其工作原理的简要版本：

* C1 层是一个卷积层，意味着它会扫描输入图像以查找其在训练期间学习到的特征。它输出一个映射，显示它在图像中看到每个学习特征的位置。这个“激活图”在 S2 层被下采样。
* C3 层是另一个卷积层，这次扫描 C1 的激活图以查找特征的*组合*。它也输出一个描述这些特征组合空间位置的激活图，该激活图在 S4 层被下采样。
* 最后，末端的全连接层 F5、F6 和 OUTPUT 是一个*分类器*，它接收最终的激活图，并将其分类到代表 10 个数字的十个类别中。

我们如何在代码中表达这个简单的神经网络？

In [ ]:
class LeNet(nn.Module): # 定义一个名为 LeNet 的类，继承自 nn.Module

    def __init__(self): # 类的初始化方法
        super(LeNet, self).__init__() # 调用父类 nn.Module 的初始化方法
        # 定义网络层
        # 第一个卷积层：输入通道数为 1（黑白图像），输出通道数为 6，卷积核大小为 3x3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 第二个卷积层：输入通道数为 6（来自上一层），输出通道数为 16，卷积核大小为 3x3
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 定义全连接层（仿射操作：y = Wx + b）
        # 第一个全连接层：输入特征数需要计算（取决于卷积和池化后的特征图大小），输出特征数为 120
        # 这里的 16 * 6 * 6 是根据 LeNet-5 针对特定输入大小（如 32x32）计算得到的特征图展平后的维度
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        # 第二个全连接层：输入特征数为 120，输出特征数为 84
        self.fc2 = nn.Linear(120, 84)
        # 第三个全连接层（输出层）：输入特征数为 84，输出特征数为 10（对应 10 个数字类别）
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x): # 定义前向传播的计算流程
        # 输入 x 通过第一个卷积层，然后应用 ReLU 激活函数，再进行最大池化（窗口大小 2x2）
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 通过第二个卷积层，应用 ReLU 激活函数，再进行最大池化（窗口大小 2x2）
        # 如果池化窗口是正方形，可以只指定一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # 将特征图展平成一维向量，以便输入到全连接层
        # x.view(-1, ...) 会自动计算第一个维度（批次大小），并将后续维度展平
        x = x.view(-1, self.num_flat_features(x))
        # 通过第一个全连接层，然后应用 ReLU 激活函数
        x = F.relu(self.fc1(x))
        # 通过第二个全连接层，然后应用 ReLU 激活函数
        x = F.relu(self.fc2(x))
        # 通过第三个全连接层（输出层），得到最终的分类分数
        x = self.fc3(x)
        return x # 返回模型的输出

    def num_flat_features(self, x): # 辅助函数，计算特征图展平后的特征数量
        size = x.size()[1:]  # 获取除批次维度外的所有维度 (C, H, W)
        num_features = 1
        for s in size: # 将所有维度的大小相乘
            num_features *= s
        return num_features

查看这段代码，您应该能够发现它与上图在结构上的一些相似之处。

这展示了一个典型 PyTorch 模型的结构：
* 它继承自 `torch.nn.Module` —— 模块可以嵌套 —— 事实上，即使是 `Conv2d` 和 `Linear` 层类也继承自 `torch.nn.Module`。
* 模型会有一个 `__init__()` 函数，在其中实例化其层，并加载可能需要的任何数据构件（例如，NLP 模型可能会加载词汇表）。
* 模型会有一个 `forward()` 函数。这是实际计算发生的地方：输入通过网络层和各种函数传递以生成输出。
* 除此之外，您可以像构建任何其他 Python 类一样构建模型类，添加支持模型计算所需的任何属性和方法。

让我们实例化这个对象，并用一个样本输入运行它。

In [ ]:
net = LeNet() # 实例化 LeNet 模型
print(net)    # 打印网络结构，显示其包含的层及其参数

# 创建一个随机的输入张量，模拟一个 32x32 的单通道（黑白）图像
# PyTorch 模型通常期望输入是批处理的，所以第一个维度是批次大小 (batch size)
# (N, C, H, W) -> (批次大小, 通道数, 高度, 宽度)
input = torch.rand(1, 1, 32, 32)
print('\n图像批次形状:')
print(input.shape)

# 将输入传递给网络进行前向传播
# 注意：我们直接调用 net(input)，而不是 net.forward(input)
# PyTorch 在背后处理了调用 forward 方法以及一些钩子函数
output = net(input)
print('\n原始输出:')
print(output) # 输出是模型对输入属于每个类别的原始分数（logits）
print(output.shape) # 输出的形状是 (批次大小, 类别数量)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Image batch shape:
torch.Size([1, 1, 32, 32])

Raw output:
tensor([[-0.0128,  0.0029, -0.0521, -0.0098, -0.0484, -0.1122, -0.0994,  0.0443,
         -0.0036, -0.0492]], grad_fn=<AddmmBackward>)
torch.Size([1, 10])


上面发生了一些重要的事情：

首先，我们实例化了 `LeNet` 类，并打印了 `net` 对象。`torch.nn.Module` 的子类将报告它创建的层及其形状和参数。如果您想了解模型的处理流程，这可以提供一个方便的概览。

下面，我们创建了一个代表 32x32 图像（1 个颜色通道）的虚拟输入。通常，您会加载一个图像块并将其转换为这种形状的张量。

您可能已经注意到我们的张量多了一个维度——*批次维度*。PyTorch 模型假定它们处理的是*批量*数据——例如，一批 16 个我们的图像块将具有形状 `(16, 1, 32, 32)`。由于我们只使用一张图像，我们创建了一个大小为 1 的批次，形状为 `(1, 1, 32, 32)`。

我们通过像调用函数一样调用模型来请求推理：`net(input)`。此调用的输出表示模型对输入代表特定数字的置信度。（由于模型的这个实例还没有学习任何东西，我们不应期望在输出中看到任何有意义的信号。）查看 `output` 的形状，我们可以看到它也有一个批次维度，其大小应始终与输入批次维度匹配。如果我们传入一个包含 16 个实例的输入批次，`output` 的形状将是 `(16, 10)`。